In [2]:
import langchain
from langchain_community.document_loaders import PyPDFLoader


In [3]:

def load_pdf(file_path):
    """Loads a PDF and extracts text from it."""
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    return pages

pdf_path = "../ikigai.pdf"
documents = load_pdf(pdf_path)


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def chunk_text(documents, chunk_size=500, chunk_overlap=100):
    """Splits extracted text into manageable chunks."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

text_chunks = chunk_text(documents)

for i, chunk in enumerate(text_chunks[:5]):
    print(f"Chunk {i+1}:\n{chunk.page_content}\n")


Chunk 1:
IKIGAI 
1. What is your reason for being? 
2. Whatever you do, don’t retire! 
3. Having a clearly defined ikigai brings satisfaction, 
happiness, and meaning to our lives. 
4. Their blood tests reveal fewer free radicals (which are 
responsible for cellular aging), as a result of drinking tea 
and eating until their stomachs are only 80 percent full 
5. the keys to longevity are diet, exercise, finding a purpose 
in life (an ikigai), and forming strong social ties—that is,

Chunk 2:
in life (an ikigai), and forming strong social ties—that is, 
having a broad circle of friends and good family relations. 
6. Members of these communities manage their time well in 
order to reduce 
7. stress, consume little meat or processed foods, and drink 
alcohol in moderation.1 They don’t do strenuous exercise, 
but they do move every day, taking walks and working in 
their vegetable gardens 
8. Fill your belly to 80 percent.” Ancient wisdom advises

Chunk 3:
their vegetable gardens 
8. Fill 

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
import google.generativeai as genai
import faiss
import numpy as np
from langchain_google_genai import GoogleGenerativeAIEmbeddings


# Initialize Gemini Embeddings
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [10]:
def generate_gemini_embeddings(text_chunks):
    """Converts text chunks into Gemini embeddings."""
    texts = [chunk.page_content for chunk in text_chunks]
    embeddings = embeddings_model.embed_documents(texts)
    return np.array(embeddings).astype("float32"), texts

# Convert text chunks to embeddings
embeddings, texts = generate_gemini_embeddings(text_chunks)


In [11]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
faiss.write_index(index, "faiss.index")


In [16]:
# Save text chunks separately
with open("stored_chunks.pkl", "wb") as f:
    pickle.dump(texts, f)  # `texts` contains the original text chunks


In [17]:
import faiss
import pickle

# Load FAISS index
index = faiss.read_index("faiss.index")

# Load stored text chunks (stored separately in a pickle file)
with open("stored_chunks.pkl", "rb") as f:
    texts = pickle.load(f)  # This should contain the chunked text


In [18]:
def extract_topics_from_chunks(chunks):
    """Extract key topics from stored FAISS chunks."""
    combined_text = "\n".join(chunks)  # Merge chunks for topic extraction

    response = genai.generate_content(
        model="gemini-pro",
        prompt=f"Extract the most important topics from the following study material:\n\n{combined_text}"
    )

    topics = response.text.split("\n")  # Convert response to list
    return topics

# Extract topics from the stored chunks
important_topics = extract_topics_from_chunks(texts)
print("\nExtracted Topics:\n", important_topics)


AttributeError: module 'google.generativeai' has no attribute 'generate_content'